In [1]:
# Import modules

import pandas as pd 
import numpy as np
import sys
import nltk
import matplotlib.pyplot as plt
import pickle

import torch
import torch.utils.data as utils
import torchwordemb

from multiprocessing import Pool

sys.path.append('/home/hugoperrin/Bureau/Data science/Kaggle/Toxic Comments/classes/')
from models import *
from data_preprocessing import text_to_caractlist, Carac2Vect

%matplotlib inline

In [2]:
path = '/home/hugoperrin/Bureau/Data science/Kaggle/Toxic Comments/data/preprocessed_data.pkl'
with open(path, 'rb') as pkl_file:
    data = pickle.load(pkl_file)
    train_comments = data['train_comments']
    test_comments = data['test_comments']
    train_labels = data['train_labels']
    del data
    
list_classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [3]:
train_comments = train_comments.reshape(train_comments.shape[0],1,train_comments.shape[1])
test_comments = test_comments.reshape(test_comments.shape[0],1,test_comments.shape[1])

In [4]:
valid_comments = train_comments[80000:,:,:]
train_comments = train_comments[:80000,:,:]

valid_labels = train_labels[80000:,:]
train_labels = train_labels[:80000,:]

In [5]:
test_comments = torch.FloatTensor(test_comments)

In [6]:
%%time

target = 'identity_hate'

labels_train = train_labels[:,list_classes.index(target)]
labels_train = labels_train.reshape(labels_train.shape[0],1)

labels_valid = valid_labels[:,list_classes.index(target)]
labels_valid = labels_valid.reshape(labels_valid.shape[0],1)

num_epoch = 5
use_GPU = True
batch_size = 512

train_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(train_comments), 
                                               torch.FloatTensor(labels_train))

valid_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(valid_comments), 
                                               torch.FloatTensor(labels_valid))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True, 
                                           num_workers = 8)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                           batch_size=batch_size,
                                           shuffle=False, 
                                           num_workers = 8)

net = CNN().cuda()
criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
optimizer = optim.RMSprop(net.parameters(), lr=0.00001, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9)

train(num_epoch, net, train_loader, optimizer, criterion, valid_loader=valid_loader)

print('\n')

>> PROCESSING LEARNING: 1511781 parameters

Epoch: 1, step:   100, training loss: 0.0765, validation loss: 0.04846
Epoch: 2, step:   200, training loss: 0.0179, validation loss: 0.04804
Epoch: 2, step:   300, training loss: 0.0507, validation loss: 0.04826
Epoch: 3, step:   400, training loss: 0.0401, validation loss: 0.04759
Epoch: 4, step:   500, training loss: 0.0111, validation loss: 0.04874
Epoch: 4, step:   600, training loss: 0.0486, validation loss: 0.05050
Epoch: 5, step:   700, training loss: 0.0339, validation loss: 0.05129


CPU times: user 22.9 s, sys: 8.04 s, total: 31 s
Wall time: 31.3 s


In [ ]:
prediction = predict(net, test_comments)

In [ ]:
# submission = pd.DataFrame()
# submission['id'] = final_id

# submission.to_csv('/home/hugoperrin/Bureau/Data science/Numerai/w90/submission2.csv', index=False)